In [ ]:
import pandas as pd
import numpy as np
# from dotenv import load_dotenv
from pandas.tseries.offsets import DateOffset
import os
import requests
import datetime
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.requests import CryptoLatestQuoteRequest
from alpaca.data.requests import CryptoTradesRequest
from alpaca.data.timeframe import TimeFrame
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
from numpy.lib import math
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Bidirectional
%matplotlib inline

## Data Retreival/Formatting

In [ ]:
# alpaca_api_key = os.getenv("ALPACA_API_KEY")
# alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
# type(alpaca_api_key)

In [ ]:
crypto_client = CryptoHistoricalDataClient()

In [ ]:
request_params = CryptoBarsRequest(
    symbol_or_symbols=["ETH/USD"],
    timeframe=TimeFrame.Minute, start='2022-06-30 00:00:00'
)

eth_bars = crypto_client.get_crypto_bars(request_params)

eth_df = eth_bars.df

eth_df

In [ ]:
eth_df.dtypes

In [ ]:
eth_df = eth_df.reset_index()
eth_df = eth_df.set_index('timestamp')
eth_df = eth_df.drop(columns=['symbol'])

eth_df

In [ ]:
# Use the pct_change function to generate  returns from close prices
eth_df["Actual Returns"] = eth_df["close"].pct_change()

# Drop na
eth_df=eth_df.dropna()

In [ ]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = eth_df[['open', 'high','low','close','volume', 'trade_count']].shift().dropna()
# Review the DataFrame
X.head()

In [ ]:
plt.plot(range(len(X)), X[['close']])

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaled = scaler.fit_transform(X)

X_scaled

### Data Split for Training

In [ ]:
# get array representation of dataframe
dataset = eth_df.values

# get number of rows to train the model on
training_data_len = math.ceil(len(dataset) * 0.8)

training_data_len

In [ ]:
# create train dataset
train_data = X_scaled[0:training_data_len, :]

# Create blank array
X_train = []
candles = 60

for i in range(candles, len(train_data)):
    X_train.append(train_data[i-candles:i, 0:6])


# create test dataset
test_data = X_scaled[training_data_len - candles: , :]

# Create blank array
X_test = []

for i in range(candles, len(test_data)):
    X_test.append(test_data[i-candles:i, 0:6])


In [ ]:
# Convert lists to array

X_train = np.array(X_train)
X_test = np.array(X_test)

display(X_train.shape)
display(len(X_train))
display(len(X_test))


In [ ]:
# Create the target set selecting the Signal column and assiging it to y
y = eth_df['Actual Returns']

# Review the value counts
display(y)

# Drop first row
# y = y.iloc[1:-candles]
y.iloc[candles+1:]

display(y)

In [ ]:
# Splitting Y data

y_train = y.iloc[0:len(X_train)]
y_test = y.iloc[len(X_train):]

display(y_train.count())
display(y_test.count())

## Model Creation

In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X.columns)

# Review the number of features
number_input_features


In [ ]:
# Create the Sequential model instance
nn = Sequential()

# Define the number of neurons in the output layer
number_output_neurons = 1

#Define the number of hidden nodes for the first hidden layer
# hidden_nodes_layer1 = (number_input_features + 1) // 2
hidden_nodes_layer1 = 64

# Define hidden nodes for all hidden layers
hidden_nodes = math.ceil(((number_input_features+1)*2)/3)
display(hidden_nodes)

# Review the number hidden nodes in the first layer
display(hidden_nodes_layer1)

#Define the number of hidden nodes for the second hidden layer
# hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) //2
hidden_nodes_layer2 = 64

# Review the number hidden nodes in the second layer
display(hidden_nodes_layer2)

In [ ]:
# Add the first hidden layer
nn.add(LSTM(units=hidden_nodes_layer1, return_sequences=True, input_shape=(candles,len(X.columns))))

In [ ]:
# # Add dropout layer
nn.add(Dropout(rate=0.2))

In [ ]:
#Add the second hidden layer
nn.add(LSTM(units=hidden_nodes_layer2, return_sequences=True))


In [ ]:
# # Add dropout layer
nn.add(Dropout(rate=0.2))

In [ ]:
# #Add the third hidden layer
nn.add(LSTM(units=hidden_nodes_layer2))

In [ ]:
# Add dropout layer
nn.add(Dropout(rate=0.2))

In [ ]:
# Add Dense layer
# nn.add(Dense(units=6, activation='relu'))

In [ ]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1,  activation='linear'))


In [ ]:
# Display the Sequential model summary, subclassed model requires build
nn.build(X_train.shape)
nn.summary()

In [ ]:
# Compile the Sequential model

nn.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Fit the model using epochs and the training data
regr_model = nn.fit(X_train, y_train, epochs=30)

In [ ]:
# Make model predictionis

y_pred=model.predict(X_test)

In [ ]:
# Plot prediction results

plt.plot(range(len(y_test)), y_test)
plt.plot(range(len(y_test)), y_pred)